In [1]:
from transformers import ClapModel, AutoProcessor
import torch
import librosa
import os
import pandas as pd
import numpy as np

In [2]:
# Load CLAP model + processor
model = ClapModel.from_pretrained("laion/larger_clap_music_and_speech")
processor = AutoProcessor.from_pretrained("laion/larger_clap_music_and_speech")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/776M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/776M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [3]:
print('total number of CLAP\'s parameters:', sum(p.numel() for p in model.parameters()))

total number of CLAP's parameters: 193913882


In [4]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('CLAP model size: {:.3f}MB'.format(size_all_mb))

CLAP model size: 740.294MB


In [5]:
#This section is to convert the uploaded zip files into folders
import zipfile

# Define the paths to your zip files
zip_files = ["Exp1.zip", "Exp2.zip", "Analysis.zip"]

# Define the directory where you want to extract the files
extract_dir = "/content/" # You can change this if you want to extract elsewhere

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract each zip file
for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")
    else:
        print(f"Erro r: {zip_file} not found.")

Extracted Exp1.zip to /content/
Extracted Exp2.zip to /content/
Extracted Analysis.zip to /content/


# Process audio

In [6]:
audio_stimuli = []
stimuli_path = "/content/Exp1/Stimuli/"

for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        audio, sample_rate = librosa.load(wav_path, sr=48000)
        audio_stimuli.append(audio)

In [7]:
inputs = processor(audios=audio_stimuli, return_tensors="pt", padding=True, sampling_rate=48000)
audio_embeddings = model.get_audio_features(**inputs)

In [8]:
print(audio_embeddings.shape)

torch.Size([59, 512])


# Process text

In [9]:
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]

discrete_captions_perceived = ["I perceive this sound as " + tag for tag in discrete_tags]
print(discrete_captions_perceived)
discrete_captions_induced = ["This sound makes me feel " + tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake"]

dimensional_captions_perceived = ["I perceive this sound as " + tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = ["This sound makes me feel " + tag for tag in dimensional_tags]
print(dimensional_captions_induced)

['I perceive this sound as happiness', 'I perceive this sound as sadness', 'I perceive this sound as anger', 'I perceive this sound as tenderness', 'I perceive this sound as fear']
['This sound makes me feel happiness', 'This sound makes me feel sadness', 'This sound makes me feel anger', 'This sound makes me feel tenderness', 'This sound makes me feel fear']
['I perceive this sound as positive', 'I perceive this sound as relaxed', 'I perceive this sound as awake']
['This sound makes me feel positive', 'This sound makes me feel relaxed', 'This sound makes me feel awake']


In [10]:
all_tags = discrete_captions_perceived + discrete_captions_induced + dimensional_captions_perceived + dimensional_captions_induced

# NOTE: currently using only dimensional_captions_induced
tag_inputs = processor(text=dimensional_captions_perceived, return_tensors="pt", padding=True)
tag_embeds = model.get_text_features(**tag_inputs)

# Generate outputs

In [11]:
sims = torch.matmul(audio_embeddings, tag_embeds.T)
print(sims.shape)

torch.Size([59, 3])


## Load csv files and extract related columns

In [12]:
IDim_path = '/content/Exp1/Data/PDim'
IDim_responses = []

all_dfs = []

for file in os.listdir(IDim_path):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(IDim_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ['positive', 'relaxed', 'awake']
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_positive_vector = []
mean_relaxed_vector = []
mean_awake_vector = []

num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_positives = []
    current_row_relaxeds = []
    current_row_awakes = []

    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_positives.append(df.iloc[i]['positive'])
        current_row_relaxeds.append(df.iloc[i]['relaxed'])
        current_row_awakes.append(df.iloc[i]['awake'])

    # Calculate the mean for the current row across all files, for each column
    mean_positive_vector.append(np.mean(current_row_positives))
    mean_relaxed_vector.append(np.mean(current_row_relaxeds))
    mean_awake_vector.append(np.mean(current_row_awakes))

IDim_responses = {
    'positive_vector': mean_positive_vector,
    'relaxed_vector': mean_relaxed_vector,
    'awake_vector': mean_awake_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of positive_vector: {len(IDim_responses['positive_vector'])}")
print(f"Length of relaxed_vector: {len(IDim_responses['relaxed_vector'])}")
print(f"Length of awake_vector: {len(IDim_responses['awake_vector'])}")

# Find the min and max values in the current sims tensor
old_min = sims.min()
old_max = sims.max()

# Define the new desired range
new_min = 1.0
new_max = 9.0

# Apply the min-max scaling formula
scaled_sims = ((sims - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims[:10]}\n")
print(f"Scaled sims min value: {scaled_sims.min():.4f}")
print(f"Scaled sims max value: {scaled_sims.max():.4f}\n")

human_ratings_tensor = torch.tensor([
    IDim_responses['positive_vector'],
    IDim_responses['relaxed_vector'],
    IDim_responses['awake_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 3)

print(f"Human ratings tensor shape: {human_ratings_tensor.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor[:10]}\n")


--- Mean Vectors (Mean across CSVs for each row position) ---
Length of positive_vector: 59
Length of relaxed_vector: 59
Length of awake_vector: 59
Scaled sims shape: torch.Size([59, 3])
Scaled sims (first 5 rows):
tensor([[4.7989, 5.6331, 5.0169],
        [5.7124, 6.3844, 6.1818],
        [3.7772, 4.6538, 4.6690],
        [2.7502, 4.1750, 4.3086],
        [7.8392, 7.6911, 7.0606],
        [6.0874, 6.2130, 5.5769],
        [3.4877, 4.0283, 3.9213],
        [3.3553, 3.9785, 3.9308],
        [4.9261, 5.7346, 5.6089],
        [3.0675, 4.4797, 4.3221]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 3])
Human ratings tensor (first 5 rows):
tensor([[3.5116, 4.4973, 3.4563],
        [4.1519, 4.9952, 3.2785],
        [4.6982, 5.4154, 4.2416],
        [5.2018, 5.2612, 4.9213],
        [3.1964, 3.4387, 3.9169],
        [3.9328, 3.8416, 4.5261],
        [4.5933, 4.8528, 4.9660],
        [4.5391, 4.6797, 5.4388]

# Evaluate

In [13]:
from scipy.stats import pearsonr # For Pearson correlation
print("Results for Dimensional Induced (PDim)")
print("______________________________________________")
# Comparison Method 1: Mean Absolute Error (MAE)
# MAE measures the average magnitude of the errors in a set of predictions, without considering their direction.
mae = torch.mean(torch.abs(scaled_sims - human_ratings_tensor))
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings: {mae:.4f}\n")

# Comparison Method 2: Mean Absolute Percentage Error (MAPE)
# MAPE measures the accuracy of a forecasting method in terms of percentage.
# Formula: MAPE = (1/n) * sum(|(Actual - Forecast) / Actual|) * 100%

# Calculate the absolute percentage error for each element
# Since human responses are between 1 and 9, division by zero is not a concern.
absolute_percentage_error = torch.abs((human_ratings_tensor - scaled_sims) / human_ratings_tensor) * 100

# Calculate the mean of these percentage errors
mape = torch.mean(absolute_percentage_error)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%\n")

# Comparison Method 3: Root Mean Squared Error (RMSE)
# RMSE measures the square root of the average of the squared differences between predicted and actual values.
# It gives a relatively high weight to large errors.
# Formula: RMSE = sqrt(mean((Actual - Forecast)^2))

# Calculate the squared differences
squared_differences = (human_ratings_tensor - scaled_sims)**2

# Calculate the mean of the squared differences (Mean Squared Error - MSE)
mse = torch.mean(squared_differences)

# Calculate the square root to get RMSE
rmse = torch.sqrt(mse)

print(f"Root Mean Squared Error (RMSE): {rmse:.4f}\n")

# Comparison Method 4: Pearson Correlation Coefficient (per column)
# Pearson correlation measures the linear relationship between two sets of data.
# We'll calculate it for each of the three columns (dimensions).

correlation_positive, _ = pearsonr(scaled_sims[:, 0].detach().numpy(), human_ratings_tensor[:, 0].numpy())
correlation_relaxed, _ = pearsonr(scaled_sims[:, 1].detach().numpy(), human_ratings_tensor[:, 1].numpy())
correlation_awake, _ = pearsonr(scaled_sims[:, 2].detach().numpy(), human_ratings_tensor[:, 2].numpy())

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings):")
print(f"  Positive Dimension: {correlation_positive:.4f}")
print(f"  Relaxed Dimension:  {correlation_relaxed:.4f}")
print(f"  Awake Dimension:    {correlation_awake:.4f}")

average_correlation = (correlation_positive + correlation_relaxed + correlation_awake) / 3
print(f"  Average Correlation: {average_correlation:.4f}")

from sklearn.metrics import r2_score

# R-squared
print("\nR-squared scores:")
r2_valence = r2_score(human_ratings_tensor[:, 0].numpy(), scaled_sims[:, 0].detach().numpy())
print(f"  valence = {r2_valence:.4f}")

r2_tension = r2_score(human_ratings_tensor[:, 1].numpy(), scaled_sims[:, 1].detach().numpy())
print(f"  tension = {r2_tension:.4f}")

r2_energy = r2_score(human_ratings_tensor[:, 2].numpy(), scaled_sims[:, 2].detach().numpy())
print(f"  energy = {r2_energy:.4f}")


Results for Dimensional Induced (PDim)
______________________________________________
Mean Absolute Error (MAE) between scaled_sims and human_ratings: 1.7495

Mean Absolute Percentage Error (MAPE): 37.24%

Root Mean Squared Error (RMSE): 2.1628

Pearson Correlation Coefficients (between scaled_sims and human_ratings):
  Positive Dimension: 0.1624
  Relaxed Dimension:  0.0564
  Awake Dimension:    -0.1692
  Average Correlation: 0.0165

R-squared scores:
  valence = -1.7973
  tension = -2.3357
  energy = -3.8419
